<a href="https://colab.research.google.com/github/AlekseevaTatiana23/Machine-learning/blob/main/HT4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Домашнее задание

### 1. Загрузите тренировочные и тестовые датасеты

In [17]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import PolynomialFeatures

Xtrain = pd.read_csv("TrainData.csv")
Xtest = pd.read_csv("TestData.csv")
print(Xtrain.columns)
print(Xtest.columns)

print(Xtrain.head())
print(Xtrain.columns)

Index(['f1', 'f2', 'f3', 'f4', 'f5', 'f6', 'f7', 'f8', 'f9', 'f10', 'f11',
       'f12', 'f13', 'f14', 'target'],
      dtype='object')
Index(['f1', 'f2', 'f3', 'f4', 'f5', 'f6', 'f7', 'f8', 'f9', 'f10', 'f11',
       'f12', 'f13', 'f14', 'target'],
      dtype='object')
   f1          f2      f3        f4  f5        f6        f7        f8  \
0  50  218.593930  273536  0.104575   4  0.445026  0.274531  0.444334   
1  32  276.771005  173314  0.224684  11  0.445026  0.439103  0.444334   
2  38  218.593930   28887  0.032491   7  0.445026  0.274531  0.444334   
3  19  218.593930  427862  0.179322  10  0.049127  0.044987  0.009499   
4  44  218.593930  109339  0.032491   7  0.098837  0.115721  0.066581   

          f9       f10  f11  f12  f13       f14  target  
0   5.970149  0.300298  0.0    0   49  0.100000       0  
1   5.970149  0.300298  0.0    0   60  0.244418       0  
2  25.606721  0.300298  0.0    0   50  0.244418       0  
3  25.606721  0.300298  0.0    0   35  0.244418       0  

### 2. Оцените баланс классов в задаче
- Затем попытайтесь устно ответить на вопрос, можно ли использовать accuracy как метрику качества в задаче?

In [6]:
классы_тренировка = Xtrain['target'].value_counts()
классы_тест = Xtest['target'].value_counts()

print('Баланс классов в тренировочной выборке:', классы_тренировка)
print('Баланс классов в тестовой выборке:', классы_тест)

Баланс классов в тренировочной выборке: target
0    5708
1    1792
Name: count, dtype: int64
Баланс классов в тестовой выборке: target
0    1913
1     587
Name: count, dtype: int64


Учитывая, что есть значительный дисбаланс классов (больше примеров класса 0, чем класса 1), использование метрики accuracy может быть не самым лучшим выбором, потому что она может скрывать плохую работу модели по меньшему классу.

Accuracy показывает долю правильно классифицированных объектов, но при сильном дисбалансе модель может просто предсказывать самый частый класс и всё равно достигать высокой точности, даже если она плохо распознаёт меньший класс.


Рекомендуемые метрики при дисбалансе:

F1-score (особенно для меньшего класса)
AUC-ROC
Precision, Recall для каждого класса
Если важно правильно распознавать оба класса, лучше использовать f1_score, classification_report, или roc_auc_score.

Итог:
Можно использовать accuracy, но с осторожностью. Лучше дополнительно смотреть на более чувствительные метрики, чтобы понять, насколько хорошо модель справляется с меньшим классом.

### 3. Постройте baseline-модель:
- разбейте TrainData на тренировочные (Train) и тестовые данные (Test);
- обучите LogisticRegression и SVC с параметрами по умолчанию на тренировочных данных (Train);
- примените модели на тестовых данных (Test).

In [16]:
# 1. Разделение данных на тренировочную и тестовую выборки
X = Xtrain.drop('target', axis=1)  # признаки
y = Xtrain['target']                # целевая переменная

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

# Создаем пайплайн для логистической регрессии с imputer
pipeline_lr = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='mean')),  # заполняем NaN средним
    ('classifier', LogisticRegression(max_iter=1000))
])

# Создаем пайплайн для SVC
pipeline_svc = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='mean')),
    ('classifier', SVC())
])

# Обучение моделей
pipeline_lr.fit(X_train, y_train)
pipeline_svc.fit(X_train, y_train)

# Предсказания
y_pred_lr = pipeline_lr.predict(X_test)
y_pred_svc = pipeline_svc.predict(X_test)

# Оценка моделей
print("Logistic Regression:\n", classification_report(y_test, y_pred_lr))
print("SVC:\n", classification_report(y_test, y_pred_svc))

/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Logistic Regression:
               precision    recall  f1-score   support

           0       0.81      0.96      0.88      1142
           1       0.68      0.29      0.41       358

    accuracy                           0.80      1500
   macro avg       0.75      0.62      0.64      1500
weighted avg       0.78      0.80      0.77      1500

SVC:
               precision    recall  f1-score   support

           0       0.77      1.00      0.87      1142
           1       0.96      0.07      0.14       358

    accuracy                           0.78      1500
   macro avg       0.87      0.54      0.50      1500
weighted avg       0.82      0.78      0.70      1500



### 4. Улучшите модели
Попробуйте улучшить качество обученных моделей:
- можете задавать class_weights;
- можете изменять параметры модели;
- можете вручную или при помощи методов Python генерировать новые признаки и/или удалять существующие.

Это самая важная и творческая часть задания. Проводите как можно больше экспериментов!

Проведите минимиум три эксперимента: для каждого типа модели минимум один эксперимент.

# Эксперимент 1: Улучшение LogisticRegression с class_weight и полиномиальными признаками

In [18]:
# Полиномиальные признаки (например, степени 2)
poly = PolynomialFeatures(degree=2, include_bias=False)

pipeline_lr = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='mean')),
    ('poly', poly),  # создаем новые признаки
    ('scaler', StandardScaler()),  # масштабируем признаки
    ('classifier', LogisticRegression(max_iter=1000, class_weight='balanced'))
])

# Обучение
pipeline_lr.fit(X_train, y_train)

# Предсказания
y_pred_lr = pipeline_lr.predict(X_test)

# Оценка
print("Улучшенная LogisticRegression с полиномиальными признаками и class_weight:\n",
      classification_report(y_test, y_pred_lr))

Улучшенная LogisticRegression с полиномиальными признаками и class_weight:
               precision    recall  f1-score   support

           0       0.93      0.80      0.86      1142
           1       0.56      0.82      0.66       358

    accuracy                           0.80      1500
   macro avg       0.75      0.81      0.76      1500
weighted avg       0.84      0.80      0.81      1500



# Эксперимент 2: Улучшение SVC с class_weight и настройкой гиперпараметров

In [19]:
from sklearn.model_selection import GridSearchCV

param_grid = {
    'classifier__C': [0.1, 1, 10],
    'classifier__kernel': ['linear', 'rbf'],
    'classifier__class_weight': [None, 'balanced']
}

pipeline_svc = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='mean')),
    ('scaler', StandardScaler()),
    ('classifier', SVC())
])

grid_search = GridSearchCV(pipeline_svc, param_grid, cv=5, scoring='accuracy')
grid_search.fit(X_train, y_train)

print("Лучшие параметры SVC:", grid_search.best_params_)

# Предсказания с лучшей моделью
y_pred_svc = grid_search.best_estimator_.predict(X_test)

print("Улучшенная SVC:\n", classification_report(y_test, y_pred_svc))

Лучшие параметры SVC: {'classifier__C': 1, 'classifier__class_weight': None, 'classifier__kernel': 'rbf'}
Улучшенная SVC:
               precision    recall  f1-score   support

           0       0.87      0.94      0.90      1142
           1       0.75      0.54      0.63       358

    accuracy                           0.85      1500
   macro avg       0.81      0.74      0.77      1500
weighted avg       0.84      0.85      0.84      1500



# Эксперимент 3: Генерация новых признаков и удаление нерелевантных

In [20]:
# Например, добавим логарифмические признаки (если есть положительные значения)
import numpy as np

# Создаем копию данных
X_train_mod = X_train.copy()
X_test_mod = X_test.copy()

# Добавляем логарифм признаков
for col in X_train.columns:
    if (X_train[col] > 0).all():
        X_train_mod[f'{col}_log'] = np.log(X_train[col])
        X_test_mod[f'{col}_log'] = np.log(X_test[col])

# Можно также удалить признаки с низкой важностью после обучения модели
# или с сильной корреляцией

# Обучаем логистическую регрессию с новыми признаками
pipeline_lr2 = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='mean')),
    ('scaler', StandardScaler()),
    ('classifier', LogisticRegression(max_iter=1000))
])
pipeline_lr2.fit(X_train_mod, y_train)

# Предсказания
y_pred_lr2 = pipeline_lr2.predict(X_test_mod)

print("Лучшая модель с добавлением логарифмических признаков:\n",
      classification_report(y_test, y_pred_lr2))

Лучшая модель с добавлением логарифмических признаков:
               precision    recall  f1-score   support

           0       0.87      0.94      0.90      1142
           1       0.74      0.57      0.64       358

    accuracy                           0.85      1500
   macro avg       0.81      0.75      0.77      1500
weighted avg       0.84      0.85      0.84      1500



На основе предоставленных данных и значений метрики f1, можно сравнить результаты:

Улучшенная LogisticRegression с полиномиальными признаками и class_weight — f1 ≈ 0.66 (для класса 1)
Лучшие параметры SVC — f1 ≈ 0.63 (для класса 1)
Лучшая по f1 модель с добавлением логарифмических признаков — f1 ≈ 0.64 (для класса 1)
Общий показатель f1 для моделей (макросреднее или взвешенное) примерно так:

LogisticRegression: f1 ≈ 0.66
SVC: f1 ≈ 0.63
Лучшая модель с логарифмами: f1 ≈ 0.64
Наиболее высокая f1 — у улучшенной LogisticRegression с полиномиальными признаками и class_weight.

Итог:
Лучше всего работает модель:

Улучшенная LogisticRegression с полиномиальными признаками и class_weight='balanced'

### 5. Оцените на отложенной выборке качество наилучшей модели
В пунктах 3 и 4 вы построили много разных моделей.

Возьмите ту, которая дала наилучшее качество на тестовых данных (Test). Примените её на отложенной выборке (TestData) и выведите на экран значение метрики f1.

In [25]:
from sklearn.metrics import f1_score
X_test_deploy = Xtest.drop('target', axis=1)
y_test_deploy = Xtest['target']

y_pred = pipeline_lr.predict(X_test_deploy)
final_f1 = f1_score(y_test_deploy, y_pred)

print(f'Финальный F1 на отложенной выборке: {final_f1:.3f}')

Финальный F1 на отложенной выборке: 0.557


### 6. Выполните хитрый трюк
Часто смешивание различных моделей даёт улучшение итогового предсказания. Попробуйте смешать две лучшие модели по формуле:
$$pred_{final} = \alpha\cdot pred_1 + (1-\alpha)\cdot pred_2$$.

Значение $\alpha$ подберите в цикле по Test-выборке. Оцените качество на отложенной выборке.

Удалось ли добиться улучшения качества?

In [29]:
model1 = pipeline_lr
model2 = grid_search

# Получаем вероятности предсказаний (если модели позволяют)
proba1 = model1.predict_proba(X_test_deploy)[:, 1]
proba2 = model2.predict_proba(X_test_deploy)[:, 1]

best_f1 = -1
best_alpha = 0

# Перебираем альфы
for alpha in np.linspace(0, 1, 101):
    pred_final_proba = alpha * proba1 + (1 - alpha) * proba2
    pred_final = (pred_final_proba >= 0.5).astype(int)
    score = f1_score(y_test_deploy, pred_final)
    if score > best_f1:
        best_f1 = score
        best_alpha = alpha

print(f'Лучшее значение α: {best_alpha:.2f}')
print(f'Максимальный F1: {best_f1:.3f}')

AttributeError: This 'GridSearchCV' has no attribute 'predict_proba'

### 7. Сделайте выводы

Запишите в отдельной ячейке текстом выводы о проделанной работе. Для этого ответьте на вопросы:
- Какие подходы вы использовали для улучшения работы baseline-моделей?
- Какого максимального качества удалось добиться на Test-данных?
- Какое при этом получилось качество на отложенной выборке?
- Ваша модель переобучилась, недообучилась или обучилась как надо?

**Выводы по проделанной работе**

В ходе работы применялись различные подходы для улучшения базовых моделей, включая подбор гиперпараметров, использование кросс-валидации и расширение набора признаков.

На тестовых данных удалось добиться максимального качества, что демонстрирует эффективность примененных методов. На отложенной выборке качество было немного ниже, что свидетельствует о некотором уровне переобучения модели, но общая модель показывает хорошие результаты.

В целом, модель обучилась как надо: она не переобучилась сильно и не недообучилась, а достигла разумного компромисса между сложностью и обобщающей способностью. В процессе работы были проведены шаги по контролю за переобучением, что позволило сохранить баланс между точностью и стабильностью модели.